In [21]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import tabulate
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import prepare
import acquire
import ml_def 
from copy import deepcopy
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
import re
rng = np.random.RandomState(0)
np.random.seed(123)
N_SPLITS = 5

# Telco dataset for modeling 

In [2]:
df = pd.read_csv(r'/Users/christophercurran/codeup-data-science/classification-exercises/telco_model.csv')
df

,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,tenure_months,tenure,churn
0,2,0,1,1,1,1,1,1,65.60,593.30,9,9.044207,False
1,1,0,0,0,2,1,0,1,59.90,542.40,9,9.055092,False
2,1,0,0,0,1,2,0,2,73.90,280.85,4,3.800406,True
3,1,1,1,0,1,2,0,2,98.00,1237.85,13,12.631122,True
4,2,1,1,0,1,2,0,1,83.90,267.40,3,3.187128,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7044,2,0,0,0,1,1,1,1,55.15,742.90,13,13.470535,False
7045,1,0,1,0,2,2,0,2,85.10,1873.70,22,22.017626,True
7046,1,0,0,0,1,1,0,1,50.30,92.75,2,1.843936,False
7047,1,0,1,1,1,1,2,1,67.85,4627.65,68,68.204127,False


In [3]:
df.columns

Index(['gender', 'is_senior_citizen', 'partner', 'dependents', 'phone_service',
       'internet_service', 'contract_type', 'payment_type', 'monthly_charges',
       'total_charges', 'tenure_months', 'tenure', 'churn'],
      dtype='object')

In [4]:
def gimmeThemStats(df):
    """
    Description
    ----
    Outputs the general statistical description of the dataframe,
    outputs the correlation heatmap, and outputs a distribution plot.
    
    Parameters
    ----
    dFrame(DataFrame):
        The dataframe for which information will be displayed.
        
    Returns
    ----
    Nothing.
    
    """
    # Description
    print("Descriptive Stats:")
    display(df.describe().T)
    
    # Heatmap
    plt.figure(figsize=(10, 8)) 
    plt.title("Heatmap", fontsize = 'x-large')
    sns.heatmap(df.corr(), annot =True)
    
    # Distribution
    ### NOTE: I changed histplot to distplot
    fig, axes = plt.subplots(4, 2, figsize=(14,14))
    fig.suptitle("Distribution Plot", y=0.92, fontsize='x-large')
    fig.tight_layout(pad=4.0)

    for i,j in enumerate(df.columns[:-1]):
        sns.distplot(df[j], ax=axes[i//2, i%2])

In [5]:
ndf = df.copy(deep = True)
ndf

,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,tenure_months,tenure,churn
0,2,0,1,1,1,1,1,1,65.60,593.30,9,9.044207,False
1,1,0,0,0,2,1,0,1,59.90,542.40,9,9.055092,False
2,1,0,0,0,1,2,0,2,73.90,280.85,4,3.800406,True
3,1,1,1,0,1,2,0,2,98.00,1237.85,13,12.631122,True
4,2,1,1,0,1,2,0,1,83.90,267.40,3,3.187128,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7044,2,0,0,0,1,1,1,1,55.15,742.90,13,13.470535,False
7045,1,0,1,0,2,2,0,2,85.10,1873.70,22,22.017626,True
7046,1,0,0,0,1,1,0,1,50.30,92.75,2,1.843936,False
7047,1,0,1,1,1,1,2,1,67.85,4627.65,68,68.204127,False


In [6]:
(ndf == 0).sum()

gender                  0
is_senior_citizen    5907
partner              3642
dependents           4934
phone_service         683
internet_service     1529
contract_type        3876
payment_type            0
monthly_charges         0
total_charges           0
tenure_months          11
tenure                 11
churn                5179
dtype: int64

In [7]:
ndf.isnull().sum()


gender                0
is_senior_citizen     0
partner               0
dependents            0
phone_service         0
internet_service      0
contract_type         0
payment_type          0
monthly_charges       0
total_charges        11
tenure_months         0
tenure                0
churn                 0
dtype: int64

In [8]:
colsToFix = ['total_charges']
ndf[colsToFix] = ndf[colsToFix].replace(np.NaN, 0)


In [9]:
ndf.isnull().sum()

gender               0
is_senior_citizen    0
partner              0
dependents           0
phone_service        0
internet_service     0
contract_type        0
payment_type         0
monthly_charges      0
total_charges        0
tenure_months        0
tenure               0
churn                0
dtype: int64

In [10]:
data= ndf.copy(deep = True)
data

,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,tenure_months,tenure,churn
0,2,0,1,1,1,1,1,1,65.60,593.30,9,9.044207,False
1,1,0,0,0,2,1,0,1,59.90,542.40,9,9.055092,False
2,1,0,0,0,1,2,0,2,73.90,280.85,4,3.800406,True
3,1,1,1,0,1,2,0,2,98.00,1237.85,13,12.631122,True
4,2,1,1,0,1,2,0,1,83.90,267.40,3,3.187128,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7044,2,0,0,0,1,1,1,1,55.15,742.90,13,13.470535,False
7045,1,0,1,0,2,2,0,2,85.10,1873.70,22,22.017626,True
7046,1,0,0,0,1,1,0,1,50.30,92.75,2,1.843936,False
7047,1,0,1,1,1,1,2,1,67.85,4627.65,68,68.204127,False


In [11]:
classifications_list = [
    LinearSVC(C= 5.0, class_weight="balanced"), SVC(kernel='rbf'), GaussianNB(), 
    KNeighborsClassifier(n_neighbors=7), DecisionTreeClassifier(max_depth=3), RandomForestClassifier(),
    ExtraTreesClassifier(), AdaBoostClassifier(), GradientBoostingClassifier()
]

In [39]:
def magic(df, target):
    """
    Description
    ----
    Splits a single given dataframe using 
    'train_test_split'
    
    
    Parameters
    ----
    df (dataframe):
        The dataframe to use for modeling.
    
    testSize (float):
        The test_size that you want to give for 
        train_test_split.
        The default test_size is set to 0.2 for 
        test_validate and test
        The default test_size is set to 0.2 for
        train and validate
    
    Returns
    ----
    save (dataframe):
        The dataframe after running the splits in
        `model_magic`
        
    """
    
    # Split
    train_validate, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, random_state=42, stratify=train_validate[target])
    
    X_train = train.drop(columns=['churn'])
    y_train = train.churn

    X_validate = validate.drop(columns=['churn'])
    y_validate = validate.churn

    X_test = test.drop(columns=['churn'])
    y_test = test.churn
    # Save
    save = model_magic(X_train, y_train, X_validate, y_validate, X_test, y_test, classifications_list)
    
    return save

In [38]:
def model_magic(X_train, y_train, X_validate, y_validate, X_test, y_test, classifierList):
    """
    Description
    ----
    This function tests out all models listed in classifierList.
    
    If the model isn't valid, the function prints out the invalid
    name of the model along with it's error.
    The accuracy score, confusion matric, positive precision, recall 
    and f-scores, and negative prevision, recall and f-scores.
    
    Data is then appended into a dictionary with columns.
    
    
    Parameters
    ----
    The X_train split for the dataframe.
        
    The X_test split for the dataframe.

    The X_validate split for dataframe.

    The y_validate splot for dataframe.

    The y_train split for the dataframe.
        
    The y_test split for the dataframe.
        
    classifierList (list of models):
        The list of models chosen.
    
    Returns
    ----
    dic (dataframe):
        A dataframe of dic.
        
    """
    # Introduce a dictionary
    dic = {'ModelName': [], 'AccuracyScore': [], 'AccuracyScoreVAL': [],
           'CorrectPredictionsCount': [], 'CorrectPredictionsCountVAL': [], 'Total': [], 'TotalVAL': [], 
           'PosPrecScore': [], 'PosPrecScoreVAL':[], 'PosRecScore': [], 'PosRecScoreVAL': [] ,'PosFScore': [],
           'PosFScoreVAL': [],'NegPrecScore': [], 'NegPrecScoreVAL': [] ,'NegRecScore': [], 'NegRecScoreVAL': [],
           'NegFScore': [], 'NegFScoreVAL': [], 'TNPercentage': [], 'TNPercentageVAL': [],'TPPercentage': [], 
           'TPPercentageVAL': [],'FNPercentage': [], 'FNPercentageVAL': [], 'FPPercentage': [], 'FPPercentageVAL': []}
    
    # Deepcopy the classifierList
    models = deepcopy(classifierList)
    
    # Test each models in the list to verify 
    for i in range(len(classifierList)):
        try:
            model = classifierList[i]
            model.fit(X_train, y_train)
        except Exception as e:
            print("==============================================================")
            print(f"I wasn't able to score with the model: {classifications_list[i]}")
            print(f"This was the error I've received from my master:\n\n{e}.")
            print("\nI didn't let it faze me though, for now I've skipped this model.")
            print("==============================================================\n")
            models.remove(classifierList[i]) # Remove invalid models from list
    
    # Loop through all models
    for classifier in range(len(models)):
        # Destroy parenthesis and anything within
        modelName = re.sub(r"\([^()]*\)", '', str(models[classifier]))
        # Performance
        model = models[classifier]
        model.fit(X_train, y_train)          
        pred = model.predict(X_test)
        pred1 = model.predict(X_validate)
        # Results
        acc_score = accuracy_score(y_test, pred)
        acc_score1 = balanced_accuracy_score(y_validate, pred1) 
        noOfCorrect = accuracy_score(y_test, pred, normalize = False)
        noOfCorrect1 = balanced_accuracy_score(y_validate, pred1, adjusted = True) 
        total = noOfCorrect/acc_score
        total1 = noOfCorrect1/acc_score1
        Confusing = confusion_matrix(y_test, pred)
        madConfusing1 = confusion_matrix(y_validate, pred1)

        dpps = Confusing[1][1] / (Confusing[1][1] + Confusing[0][1]) # pos prec score
        dpps1 = madConfusing1[1][1] / (madConfusing1[1][1] + madConfusing1[0][1])
        dprs = Confusing[1][1] / (Confusing[1][1] + Confusing[1][0]) # pos rec score
        dprs1 = madConfusing1[1][1] / (madConfusing1[1][1] + madConfusing1[1][0])
        dpfs = 2 * (dpps * dprs) / (dpps + dprs) # pos f1 score
        dpfs1 = 2 * (dpps1 * dprs1) / (dpps1 + dprs1) # pos f1 score
        dnps = Confusing[0][0] / (Confusing[0][0] + Confusing[1][0]) # neg prec score
        dnps1 = madConfusing1[0][0] / (madConfusing1[0][0] + madConfusing1[1][0])
        dnrs = Confusing[0][0] / (Confusing[0][0] + Confusing[0][1]) # neg rec score
        dnrs1 = madConfusing1[0][0] / (madConfusing1[0][0] + madConfusing1[0][1])
        dnfs = 2 * (dnps * dnrs) / (dnps + dnrs) # neg f1 score
        dnfs1 = 2 * (dnps1 * dnrs1) / (dnps1 + dnrs1) 


        # Save everything
        dic['ModelName'].append(modelName)
        dic['AccuracyScore'].append(acc_score)
        dic['AccuracyScoreVAL'].append(acc_score1)
        dic['CorrectPredictionsCount'].append(noOfCorrect)
        dic['CorrectPredictionsCountVAL'].append(noOfCorrect1)
        dic['Total'].append(total)
        dic['TotalVAL'].append(total1)
        dic['PosPrecScore'].append(dpps)
        dic['PosPrecScoreVAL'].append(dpps1)
        dic['PosRecScore'].append(dprs)
        dic['PosRecScoreVAL'].append(dprs1)
        dic['PosFScore'].append(dpfs)
        dic['PosFScoreVAL'].append(dpfs1)
        dic['NegPrecScore'].append(dnps)
        dic['NegPrecScoreVAL'].append(dnps1)
        dic['NegRecScore'].append(dnrs)
        dic['NegRecScoreVAL'].append(dnrs1)
        dic['NegFScore'].append(dnfs)
        dic['NegFScoreVAL'].append(dnfs1)
        dic['TNPercentage'].append(Confusing[0][0]/total*100)
        dic['TNPercentageVAL'].append(madConfusing1[0][0]/total*100)
        dic['TPPercentage'].append(Confusing[1][1]/total*100)
        dic['TPPercentageVAL'].append(madConfusing1[1][1]/total*100)
        dic['FNPercentage'].append(Confusing[1][0]/total*100)
        dic['FNPercentageVAL'].append(madConfusing1[1][0]/total*100)
        dic['FPPercentage'].append(Confusing[0][1]/total*100)
        dic['FPPercentageVAL'].append(madConfusing1[0][1]/total*100)
        
    return pd.DataFrame.from_dict(dic)

In [40]:
magic(data, 'churn')

,ModelName,AccuracyScore,AccuracyScoreVAL,CorrectPredictionsCount,CorrectPredictionsCountVAL,Total,TotalVAL,PosPrecScore,PosPrecScoreVAL,PosRecScore,...,NegFScore,NegFScoreVAL,TNPercentage,TNPercentageVAL,TPPercentage,TPPercentageVAL,FNPercentage,FNPercentageVAL,FPPercentage,FPPercentageVAL
0,LinearSVC,0.755319,0.568267,1065,0.136535,1410.0,0.240265,0.795918,0.835443,0.104278,...,0.856070,0.861345,72.765957,87.234043,2.765957,4.680851,23.758865,27.163121,0.709220,0.921986
1,SVC,0.734752,0.500000,1036,0.000000,1410.0,0.000000,NaN,NaN,0.000000,...,0.847097,0.847019,73.475177,88.156028,0.000000,0.000000,26.524823,31.843972,0.000000,0.000000
2,GaussianNB,0.739007,0.767720,1042,0.535440,1410.0,0.697442,0.505556,0.547049,0.729947,...,0.806925,0.831023,54.539007,68.014184,19.361702,24.326241,7.163121,7.517730,18.936170,20.141844
3,KNeighborsClassifier,0.762411,0.675899,1075,0.351798,1410.0,0.520489,0.570397,0.607670,0.422460,...,0.845551,0.855162,65.035461,78.723404,11.205674,14.609929,15.319149,17.234043,8.439716,9.432624
4,DecisionTreeClassifier,0.790780,0.670988,1115,0.341976,1410.0,0.509660,0.690821,0.704280,0.382353,...,0.868245,0.871546,68.936170,82.765957,10.141844,12.836879,16.382979,19.007092,4.539007,5.390071
5,RandomForestClassifier,0.777305,0.702316,1096,0.404632,1410.0,0.576140,0.604895,0.636111,0.462567,...,0.854630,0.863689,65.460993,78.865248,12.269504,16.241135,14.255319,15.602837,8.014184,9.290780
6,ExtraTreesClassifier,0.773050,0.698539,1090,0.397078,1410.0,0.568441,0.588235,0.599490,0.481283,...,0.850467,0.854109,64.539007,77.021277,12.765957,16.666667,13.758865,15.177305,8.936170,11.134752
7,AdaBoostClassifier,0.792908,0.714290,1118,0.428581,1410.0,0.600009,0.641379,0.688623,0.497326,...,0.864564,0.875817,66.099291,80.780142,13.191489,16.312057,13.333333,15.531915,7.375887,7.375887
8,GradientBoostingClassifier,0.797163,0.718436,1124,0.436871,1410.0,0.608087,0.646667,0.695522,0.518717,...,0.866729,0.877692,65.957447,80.921986,13.758865,16.524823,12.765957,15.319149,7.517730,7.234043
